In [1]:
from flask import Flask, render_template, jsonify, request, session, g
import numpy as np
import joblib
import pickle
from tensorflow.keras.models import load_model
from datetime import datetime, timedelta
import random
import time
import threading
import warnings
import os
import absl.logging
from werkzeug.utils import secure_filename
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Suppress TensorFlow warnings
absl.logging.set_verbosity(absl.logging.ERROR)
warnings.filterwarnings('ignore', category=UserWarning)

app = Flask(__name__)
app.secret_key = 'your_secret_key_here'  # Required for session

# Configuration for file uploads
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'pkl', 'h5', 'joblib'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure necessary directories exist
for folder in ['models', 'uploads', 'static/css']:
    if not os.path.exists(folder):
        os.makedirs(folder)

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Load models at startup
models = {
    'scaler': None,
    'autoencoder': None,
    'autoencoder_threshold': None,
    'kmeans': None,
    'kmeans_threshold': None,
    'isolation_forest': None,
    'feature_cols': None
}

# Training history and active model
training_history = []
active_model = 'autoencoder'  # Default active model

# System settings storage
system_settings = {
    'interface': 'eth0',
    'capture_mode': 'promiscuous',
    'alert_threshold': 'high',
    'data_retention': '30 days',
    'email_alerts': 'enabled',
    'email_address': 'elphordmachida@gmail.com',
    'ui_theme': 'light',
    'email_settings': {
        'smtp_server': 'smtp.gmail.com',
        'smtp_port': 587,
        'sender_email': 'elphordkudzai15@gmail.com',
        'sender_password': 'xcnu vxwb bgrq ehde',  # Will be set through settings
        'receiver_email': 'elphordmachida@gmail.com'
    }
}

@app.before_request
def before_request():
    # Load settings for every request
    g.settings = {
        'ui_theme': session.get('ui_theme', system_settings.get('ui_theme', 'light')),
        'timezone': session.get('timezone', 'UTC'),
        'refresh_interval': session.get('refresh_interval', 5)
    }
    
    # Sync session theme with system settings
    if 'ui_theme' not in session:
        session['ui_theme'] = system_settings['ui_theme']

@app.context_processor
def inject_theme():
    return {'theme_class': 'dark-theme' if g.settings['ui_theme'] == 'dark' else ''}

def load_models():
    """Load all the trained models"""
    try:
        print("Loading models...")
        models['scaler'] = joblib.load('models/scaler.pkl')
        models['autoencoder'] = load_model('models/autoencoder_model.h5')
        models['kmeans'] = joblib.load('models/kmeans_model.pkl')
        models['isolation_forest'] = joblib.load('models/isolation_forest_model.pkl')
        
        with open('models/autoencoder_threshold.pkl', 'rb') as f:
            models['autoencoder_threshold'] = pickle.load(f)
            
        with open('models/kmeans_threshold.pkl', 'rb') as f:
            models['kmeans_threshold'] = pickle.load(f)
            
        # These should match the features used during training
        models['feature_cols'] = [
            'packet_size', 'protocol', 'src_port', 'dst_port',
            'tcp_fin', 'tcp_syn', 'tcp_rst', 'tcp_psh', 'tcp_ack', 'tcp_urg'
        ]
        
        print("All models loaded successfully")
    except Exception as e:
        print(f"Error loading models: {str(e)}")
        # Create dummy models for demonstration if real models fail to load
        models['feature_cols'] = [
            'packet_size', 'protocol', 'src_port', 'dst_port',
            'tcp_fin', 'tcp_syn', 'tcp_rst', 'tcp_psh', 'tcp_ack', 'tcp_urg'
        ]

# Load models when app starts
load_models()

# Simulated data storage
traffic_data = {
    'timestamps': [],
    'packet_counts': [],
    'protocol_distribution': {
        'TCP': 0,
        'UDP': 0,
        'ICMP': 0,
        'HTTP': 0,
        'HTTPS': 0,
        'Other': 0
    },
    'anomalies': [],
    'alerts': []
}

# Counter for generating unique IDs
alert_id_counter = 1000
anomaly_id_counter = 1000

def send_alert_email(alert):
    """Send email notification for high severity alerts"""
    if system_settings['email_alerts'] != 'enabled':
        return False
    
    try:
        # Create message
        msg = MIMEMultipart()
        msg['From'] = system_settings['email_settings']['sender_email']
        msg['To'] = system_settings['email_settings']['receiver_email']
        msg['Subject'] = f"NMRL NIDS Alert: {alert['severity'].upper()} - {alert['message']}"
        
        # Email body
        body = f"""
        New Security Alert Detected:
        
        Severity: {alert['severity'].upper()}
        Time: {alert['timestamp']}
        Message: {alert['message']}
        Alert ID: {alert['id']}
        
        Details:
        Source IP: {alert.get('details', {}).get('source_ip', 'N/A')}
        Destination IP: {alert.get('details', {}).get('destination_ip', 'N/A')}
        Anomaly Type: {alert.get('details', {}).get('anomaly_type', 'N/A')}
        
        Please log in to the NMRL NIDS dashboard for more details.
        """
        
        msg.attach(MIMEText(body, 'plain'))
        
        # Connect to SMTP server and send
        with smtplib.SMTP(
            system_settings['email_settings']['smtp_server'],
            system_settings['email_settings']['smtp_port']
        ) as server:
            server.starttls()
            server.login(
                system_settings['email_settings']['sender_email'],
                system_settings['email_settings']['sender_password']
            )
            server.send_message(msg)
        
        return True
    except Exception as e:
        print(f"Error sending email alert: {str(e)}")
        return False

# Simulate network traffic updates
def simulate_network_traffic():
    """Background thread to simulate network traffic data"""
    protocols = ['TCP', 'UDP', 'ICMP', 'HTTP', 'HTTPS', 'Other']
    global alert_id_counter, anomaly_id_counter
    
    while True:
        try:
            # Update timestamps (last 30 minutes)
            now = datetime.now()
            traffic_data['timestamps'] = [(now - timedelta(minutes=i)).strftime('%H:%M') 
                                         for i in range(30, -1, -1)]
            
            # Generate random packet counts
            traffic_data['packet_counts'] = [random.randint(50, 200) for _ in range(31)]
            
            # Update protocol distribution
            for protocol in protocols:
                traffic_data['protocol_distribution'][protocol] = random.randint(5, 50)
            
            # Generate some random anomalies (5-10% chance)
            if random.random() < 0.05:
                anomaly_types = ['Port Scan', 'DDoS', 'Malware', 'Brute Force', 'Data Exfiltration']
                anomaly_id_counter += 1
                anomaly = {
                    'id': str(anomaly_id_counter),
                    'timestamp': now.strftime('%H:%M:%S'),
                    'anomaly_type': random.choice(anomaly_types),
                    'src_ip': f"192.168.{random.randint(1,255)}.{random.randint(1,255)}",
                    'anomaly_score': round(random.uniform(0.7, 1.0), 2),
                    'severity': random.choice(['low', 'medium', 'high']),
                    'status': 'detected'
                }
                traffic_data['anomalies'].append(anomaly)
                
                # Generate alert for high score anomalies
                if anomaly['anomaly_score'] > 0.9:
                    alert_id_counter += 1
                    alert = {
                        'id': str(alert_id_counter),
                        'timestamp': now.strftime('%H:%M:%S'),
                        'severity': 'high',
                        'message': f"{anomaly['anomaly_type']} detected from {anomaly['src_ip']}",
                        'status': 'active',
                        'details': {
                            'source_ip': anomaly['src_ip'],
                            'anomaly_type': anomaly['anomaly_type'],
                            'anomaly_score': anomaly['anomaly_score'],
                            'detected_by': active_model
                        }
                    }
                    traffic_data['alerts'].append(alert)
                    
                    # Send email notification for high severity alerts
                    if alert['severity'] == 'high' and system_settings['email_alerts'] == 'enabled':
                        send_alert_email(alert)
            
            # Keep only recent data
            traffic_data['anomalies'] = traffic_data['anomalies'][-20:]
            traffic_data['alerts'] = traffic_data['alerts'][-20:]
            
            time.sleep(5)
        except Exception as e:
            print(f"Error in traffic simulation: {str(e)}")
            time.sleep(1)

# Start the background thread
traffic_thread = threading.Thread(target=simulate_network_traffic)
traffic_thread.daemon = True
traffic_thread.start()

@app.route('/api/models/import', methods=['POST'])
def import_model():
    """API endpoint to import model files"""
    if 'file' not in request.files:
        return jsonify({'status': 'error', 'message': 'No file part'}), 400
    
    file = request.files['file']
    if file.filename == '':
        return jsonify({'status': 'error', 'message': 'No selected file'}), 400
    
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        
        try:
            # Try to load the model based on file extension
            ext = filename.rsplit('.', 1)[1].lower()
            model_name = filename.split('.')[0]  # Get model name from filename
            
            if ext == 'h5':
                # Keras model
                model = load_model(filepath)
                models[model_name] = model
            elif ext in ['pkl', 'joblib']:
                # Scikit-learn model or other pickle file
                if ext == 'pkl':
                    with open(filepath, 'rb') as f:
                        model = pickle.load(f)
                else:
                    model = joblib.load(filepath)
                models[model_name] = model
            
            return jsonify({
                'status': 'success',
                'message': f'Model {filename} imported and loaded successfully',
                'model_name': model_name
            })
        except Exception as e:
            return jsonify({
                'status': 'error',
                'message': f'Failed to load model: {str(e)}'
            }), 500
    
    return jsonify({'status': 'error', 'message': 'Invalid file type'}), 400

@app.route('/api/models/activate', methods=['POST'])
def activate_model():
    """API endpoint to activate a specific model"""
    global active_model
    model_type = request.json.get('model_type')
    if model_type in ['autoencoder', 'kmeans', 'isolation_forest']:
        active_model = model_type
        return jsonify({'status': 'success', 'active_model': active_model})
    return jsonify({'status': 'error', 'message': 'Invalid model type'}), 400

@app.route('/api/models/train', methods=['POST'])
def train_model():
    """API endpoint to train models"""
    try:
        model_type = request.json.get('model_type')
        training_data = request.json.get('training_data')
        hyperparams = request.json.get('hyperparameters', {})
        
        # Simulate training (in a real app, you'd call your actual training code)
        training_time = random.randint(2, 10)
        time.sleep(training_time)
        
        # Record training
        training_history.append({
            'date': datetime.now().strftime('%Y-%m-%d %H:%M'),
            'model': model_type,
            'duration': f"{training_time}m {random.randint(0,59)}s",
            'status': 'Complete',
            'performance': {
                'accuracy': round(random.uniform(0.8, 0.95), 2),
                'precision': round(random.uniform(0.75, 0.9), 2),
                'recall': round(random.uniform(0.8, 0.93), 2)
            }
        })
        
        return jsonify({
            'status': 'success',
            'training_id': len(training_history),
            'duration': training_time
        })
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

@app.route('/api/models/history')
def get_training_history():
    """API endpoint to get training history"""
    return jsonify(training_history[-10:][::-1])  # Return last 10 entries, newest first

@app.route('/api/anomalies/<anomaly_id>/block', methods=['POST'])
def block_anomaly(anomaly_id):
    try:
        # In a real app, you'd implement actual blocking logic
        # For our mock data, we'll just mark it as blocked
        for anomaly in traffic_data['anomalies']:
            if anomaly.get('id') == anomaly_id:
                anomaly['status'] = 'blocked'
                break
                
        return jsonify({'status': 'success'})
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

@app.route('/api/anomalies/filter', methods=['GET'])
def filter_anomalies():
    try:
        severity = request.args.get('severity')
        anomaly_type = request.args.get('type')
        time_range = request.args.get('time_range', '24h')
        
        # Filter mock data
        filtered = traffic_data['anomalies'].copy()
        
        if severity:
            filtered = [a for a in filtered if a.get('severity') == severity]
            
        if anomaly_type:
            filtered = [a for a in filtered if a.get('anomaly_type') == anomaly_type]
            
        if time_range == '24h':
            # Filter for last 24 hours in real app
            pass
        elif time_range == '7d':
            # Filter for last 7 days in real app
            pass
            
        return jsonify(filtered[::-1])  # Return newest first
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

@app.route('/api/alerts/<alert_id>/resend_email', methods=['POST'])
def resend_alert_email(alert_id):
    """API endpoint to resend alert email"""
    try:
        alert = next((a for a in traffic_data['alerts'] if a.get('id') == alert_id), None)
        if alert:
            success = send_alert_email(alert)
            if success:
                return jsonify({'status': 'success'})
            return jsonify({'status': 'error', 'message': 'Failed to send email'}), 500
        return jsonify({'status': 'error', 'message': 'Alert not found'}), 404
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

# Route definitions
@app.route('/')
def dashboard():
    """Render the main dashboard"""
    settings = {
        'preferred_model': active_model,  # Use the active model
        'system_status': 'active',
        'last_updated': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }

    alert_counts = {
        'high': sum(1 for alert in traffic_data['alerts'] if alert['severity'] == 'high'),
        'medium': sum(1 for alert in traffic_data['alerts'] if alert['severity'] == 'medium'),
        'low': sum(1 for alert in traffic_data['alerts'] if alert['severity'] == 'low')
    }
    
    return render_template('index.html',
                         total_packets=sum(traffic_data['packet_counts']),
                         total_anomalies=len(traffic_data['anomalies']),
                         alert_counts=alert_counts,
                         settings=settings,
                         recent_anomalies=traffic_data['anomalies'][-5:][::-1],
                         recent_alerts=traffic_data['alerts'][-5:][::-1],
                         theme_settings=g.settings)

@app.route('/traffic_monitor')
def traffic_monitor():
    """Traffic monitoring page"""
    return render_template('traffic.html',
                         traffic_stats=traffic_data,
                         active_page='traffic_monitor',
                         theme_settings=g.settings)

@app.route('/anomalies')
def anomalies():
    """Anomalies page"""
    return render_template('anomalies.html',
                         anomalies=traffic_data['anomalies'][::-1],  # Show newest first
                         active_page='anomalies',
                         theme_settings=g.settings)

@app.route('/alerts')
def alerts():
    """Alerts page"""
    return render_template('alerts.html',
                         alerts=traffic_data['alerts'][::-1],  # Show newest first
                         active_page='alerts',
                         theme_settings=g.settings)

@app.route('/model_management')
def model_management():
    """Model management page"""
    model_info = {
        'autoencoder': {
            'status': 'active' if models['autoencoder'] else 'inactive',
            'last_trained': '2025-04-28',
            'performance': {'accuracy': 0.92, 'precision': 0.88, 'recall': 0.90}
        },
        'kmeans': {
            'status': 'active' if models['kmeans'] else 'inactive',
            'last_trained': '2025-04-28',
            'performance': {'accuracy': 0.85, 'precision': 0.82, 'recall': 0.83}
        },
        'isolation_forest': {
            'status': 'active' if models['isolation_forest'] else 'inactive',
            'last_trained': '2025-04-28',
            'performance': {'accuracy': 0.89, 'precision': 0.87, 'recall': 0.88}
        }
    }
    return render_template('models.html',
                         model_info=model_info,
                         active_page='model_management',
                         training_history=training_history[-10:][::-1],
                         theme_settings=g.settings)

@app.route('/system_settings_page')
def system_settings_page():
    """System settings page"""
    return render_template('settings.html',
                         settings=system_settings,
                         active_page='system_settings',
                         theme_settings=g.settings)

# API endpoints
@app.route('/api/traffic/stats')
def traffic_stats():
    """API endpoint for traffic statistics"""
    return jsonify({
        'traffic_over_time': {
            'labels': traffic_data['timestamps'],
            'values': traffic_data['packet_counts']
        },
        'protocol_distribution': traffic_data['protocol_distribution']
    })

@app.route('/api/anomalies/recent')
def recent_anomalies():
    """API endpoint for recent anomalies"""
    count = int(request.args.get('count', 5))
    return jsonify(traffic_data['anomalies'][-count:][::-1])

@app.route('/api/alerts/recent')
def recent_alerts():
    """API endpoint for recent alerts"""
    count = int(request.args.get('count', 5))
    return jsonify(traffic_data['alerts'][-count:][::-1])

@app.route('/api/alerts/<alert_id>/resolve', methods=['POST'])
def resolve_alert(alert_id):
    """API endpoint to resolve a specific alert"""
    try:
        # In a real app, you'd update your database here
        # For our mock data, we'll just mark it as resolved
        for alert in traffic_data['alerts']:
            if alert.get('id') == alert_id:
                alert['status'] = 'resolved'
                break
                
        return jsonify({'status': 'success'})
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

@app.route('/api/alerts/<alert_id>')
def get_alert_details(alert_id):
    """API endpoint to get details for a specific alert"""
    # Find the alert in our mock data
    alert = next((a for a in traffic_data['alerts'] if a.get('id') == alert_id), None)
    if alert:
        return jsonify(alert)
    return jsonify({'status': 'error', 'message': 'Alert not found'}), 404

@app.route('/api/system/status')
def system_status():
    """API endpoint for system status"""
    return jsonify({
        'status': 'active',
        'models_loaded': bool(models['autoencoder'] and models['kmeans'] and models['isolation_forest']),
        'last_updated': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    })

@app.route('/api/settings/save', methods=['POST'])
def save_settings():
    """API endpoint to save system settings"""
    try:
        new_settings = request.json
        # Update both system settings and session
        global system_settings
        
        # Handle email password separately (don't overwrite if empty)
        if 'email_settings' in new_settings:
            if 'sender_password' in new_settings['email_settings']:
                if new_settings['email_settings']['sender_password'] == '':
                    # Keep existing password if new one is empty
                    new_settings['email_settings']['sender_password'] = system_settings['email_settings']['sender_password']
        
        system_settings.update(new_settings)
        
        # Update session settings
        if 'ui_theme' in new_settings:
            session['ui_theme'] = new_settings['ui_theme']
        if 'timezone' in new_settings:
            session['timezone'] = new_settings['timezone']
        if 'refresh_interval' in new_settings:
            session['refresh_interval'] = new_settings['refresh_interval']
            
        return jsonify({'status': 'success'})
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

@app.route('/api/settings/check_updates')
def check_updates():
    """API endpoint to check for system updates"""
    # Simulate update check
    return jsonify({
        'status': 'success',
        'update_available': random.choice([True, False]),
        'latest_version': '1.3.0',
        'current_version': '1.2.5'
    })

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)

Loading models...
All models loaded successfully
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.9:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 23:54:45] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 23:54:46] "GET /api/traffic/stats HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 23:54:46] "GET /api/anomalies/recent?count=5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 23:54:46] "GET /api/alerts/recent?count=5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 23:54:51] "GET /api/traffic/stats HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 23:54:51] "GET /api/anomalies/recent?count=5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 23:54:51] "GET /api/alerts/recent?count=5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 23:54:56] "GET /api